In [183]:
import asyncio
import websockets
import json
import pandas as pd
from csv import writer
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np
import time
from pandas._libs.window.aggregations import ewma
import time

from model import *
from utils import *

In [184]:
# window size
WINDOW_SIZE = 100

# features
FEATURES = ['bid_diff_feature_1', 'ask_diff_feature_1']

# ewma params
COM = 10
GAMMA = 20

# lob depth
DEPTH = 10

In [185]:
msg = \
{
  "jsonrpc" : "2.0",
  "id" : 8772,
  "method" : "public/get_order_book",
  "params" : {
    "instrument_name" : "BTC-PERPETUAL",
    "depth" : DEPTH,
  }
}


In [ ]:
model_name = "20210714_000000"

In [186]:
model = build_model(window_size=WINDOW_SIZE, n_features=len(FEATURES))
model.load_weights(f'_output/models/{model_name}.h5')

# This should be reworked since the input is now different for the predictor

In [ ]:
X1 = np.zeros((1, 100, 40, 1))
X2 = np.zeros((1, 100, 2))

predictions = np.zeros(100)

best_bid_price, best_ask_price = 0, 0

pnl = 0
money = 0
holdings = 0
fees = 0
trades = 0

async with websockets.connect('wss://www.deribit.com/ws/api/v2') as websocket:
    
    i = 0
    
    t_start = time.time()
    
    print(f"{'time':12}{'latency':12}{'iter':12}{'bid':12}{'ask':12}{'trades':12}{'position':12}{'pnl':12}{'fees':12}")
    
    while True:
        
        t_iter_start = time.time()

        await websocket.send(json.dumps(msg))
        
        data = await websocket.recv()
        data = json.loads(data)
        
        new_best_bid_price = data['result']['best_bid_price']
        new_best_ask_price = data['result']['best_ask_price']

        l = 10.
        u = 1000000.

        aps = np.array([a[0] for a in data['result']['asks']]) - new_best_ask_price
        avs = (np.array([a[1] for a in data['result']['asks']]) - l) / (u - l)
        bps = new_best_bid_price - np.array([b[0] for b in data['result']['bids']])
        bvs = (np.array([b[1] for b in data['result']['bids']]) - l) / (u - l)
                
        X1 = np.roll(X1, -1, axis=1)
        X1[0, -1, :, 0] = [x for items in zip(aps, bps, avs, bvs) for x in items]
        
        X2 = np.roll(X2, -1, axis=1)
        X2[0, -1, :] = [new_best_bid_price - best_bid_price, best_ask_price - new_best_ask_price]
        
        best_bid_price = new_best_bid_price
        best_ask_price = new_best_ask_price
        
        predictions = np.roll(predictions, -1)
        predictions[-1] = GAMMA * (np.argmax(model.predict([X1, X2]), axis=1) - 1.)
        
        strategy = np.minimum(1., np.maximum(-1., np.round(ewma(predictions, COM, True, True, 1))))[-1]
        
        if i > 100:
        
            diff = strategy - holdings
            holdings += diff
            trades += abs(diff) > 0

            # trade
            money_diff = np.maximum(diff, 0) * best_ask_price + np.minimum(diff, 0) * best_bid_price
            money -= money_diff
            fees += abs(money_diff) * 0.0005
            
            pnl = money + holdings * (best_ask_price + best_bid_price) / 2
        
        t_elapsed = time.time() - t_start
        t_iter_elapsed = 1000 * (time.time() - t_iter_start)
        
        output = f"{int(t_elapsed//60//60%24):02}:{int(t_elapsed//60%60):02}:{int(t_elapsed%60):02}" + " "*4
        output += f"{int(t_iter_elapsed):3}ms" + " "*7
        output += f"{i:<12}"
        output += f"{best_bid_price:<12.2f}"
        output += f"{best_ask_price:<12.2f}"
        output += f"{trades:<12}"
        output += f"{holdings:<12}"
        output += f"{pnl:<12.2f}"
        output += f"{fees:.2f}"
        output += "\r"

        print(output, end='')
        
        i += 1